In [1]:
import pandas as pd

In [2]:
df = pd.read_csv(r"C:\Users\aswat\OneDrive\Desktop\ajesh\Artificial Intelligence\ICT Academy\datas\titanic.csv")

In [3]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
x = df[['Pclass','SibSp','Parch','Fare']]
y = df.Survived

**Scaling**

In [5]:
from sklearn.preprocessing import StandardScaler
sc= StandardScaler()
x=sc.fit_transform(x)

**Splitting**

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

# SVM

In [8]:
from sklearn.svm import SVC
model=SVC(kernel='linear',C=1,gamma='auto')
model.fit(X_train,y_train)

SVC(C=1, gamma='auto', kernel='linear')

In [10]:
y_pred = model.predict(X_test) 

In [11]:
from sklearn.metrics import accuracy_score
accuracy_score(y_pred,y_test)

0.6502242152466368

In [12]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_pred,y_test)

array([[94, 31],
       [47, 51]], dtype=int64)

# 5-Fold Cross Validation

In [15]:
from sklearn.model_selection import cross_val_score,KFold
model = SVC(kernel='linear',C=1,gamma='auto')
kf=KFold(n_splits=5)
score=cross_val_score(model,x,y,cv=kf)
print("Cross Validation Scores are {}".format(score))

Cross Validation Scores are [0.62011173 0.61235955 0.66292135 0.7247191  0.69101124]


In [16]:
print("Average Cross Validation score :{}".format(score.mean()))

Average Cross Validation score :0.6622245935597263


# Random Search

In [17]:
parameters={'C': [0.1, 1, 10, 100, 1000], 
'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
'kernel': ['rbf']
}

In [18]:
from sklearn.model_selection import RandomizedSearchCV
random_search=RandomizedSearchCV(estimator = SVC(), param_distributions=parameters,verbose=1, n_jobs=-1,
                            n_iter=50)
random_result = random_search.fit(X_train, y_train) 
print('Best Score: ', random_result.best_score_*100)
print('Best Params: ', random_result.best_params_)

C:\Python310\lib\site-packages\sklearn\model_selection\_search.py:306: UserWarning: The total space of parameters 25 is smaller than n_iter=50. Running 25 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 5 folds for each of 25 candidates, totalling 125 fits
Best Score:  72.90090898889014
Best Params:  {'kernel': 'rbf', 'gamma': 0.01, 'C': 1000}


# Hyperopt

In [33]:
from hyperopt import tpe, hp, fmin, STATUS_OK,Trials

In [36]:
space = {
    'C': hp.choice('C',  [0.1, 1, 10, 100, 1000]),
   'kernel': hp.choice('kernel',['rbf'] ),
    'gamma': hp.choice('gamma', [1, 0.1, 0.01, 0.001, 0.0001]),
}

In [41]:
def hyperparameter_tuning(params):
    clf = SVC(**params)
    acc = cross_val_score(clf, x, y,scoring="accuracy").mean()
    return {"loss": -acc, "status": STATUS_OK}

In [42]:
trials = Trials()

best = fmin(
    fn=hyperparameter_tuning,
    space = space, 
    algo=tpe.suggest, 
    max_evals=50, 
    trials=trials
)

print("Best: {}".format(best))

100%|███████████████████████████████████████████████| 50/50 [00:06<00:00,  7.67trial/s, best loss: -0.7150524135333626]
Best: {'C': 1, 'gamma': 1, 'kernel': 0}
